# Práctica 1: Web Scraping
## Asignatura de Tipología y ciclo de vida de los datos de la Universidad Oberta de Catalunya (UOC)

### por Juan Alonso Franco Blanco y Juan Prieto-Pena

### Primera parte: Obtención de las URL mediante el sitemap de la página.

La primera parte de la práctica consistirá en obtener las url de las páginas web de las que vamos a sacar la información. Para ello, descargaremos el fichero xml del sitemap de la página web.

Emplearemos las siguientes librerías:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

Antes de comenzar, cambiaremos los headers para evitar que se nos identifique como script y evitar ser bloqueados.

In [2]:
headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\*/*;q=0.8", 
           "Accept-Encoding": "gzip, deflate, sdch, br",
           "Accept-Language": "en-US,en;q=0.8",
           "Cache-Control": "no-cache",
           "dnt": "1",
           "Pragma": "no-cache",
           "Upgrade-Insecure-Requests": "1",
           "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
}

Extraemos directamente la información en formato xml del sitemap. Cargamos la información del sitio usando BeautifulSoup:

In [3]:
r = requests.get("https://preciosmundi.com/sitemap.xml",headers=headers) #Obtenemos la información del sitemap
xml = r.text
soup = BeautifulSoup(xml,'xml') #Creamos un objeto soup para facilitar la navegación del xml
soup

<?xml version="1.0" encoding="utf-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.sitemaps.org/schemas/sitemap/0.9             http://www.sitemaps.org/schemas/sitemap/0.9/sitemap.xsd">
<url>
<loc>https://preciosmundi.com/</loc>
</url>
<url>
<loc>https://preciosmundi.com/europa/</loc>
</url>
<url>
<loc>https://preciosmundi.com/america/</loc>
</url>
<url>
<loc>https://preciosmundi.com/africa/</loc>
</url>
<url>
<loc>https://preciosmundi.com/asia/</loc>
</url>
<url>
<loc>https://preciosmundi.com/oceania/</loc>
</url>
<url>
<loc>https://preciosmundi.com/comparar</loc>
</url>
<url>
<loc>https://preciosmundi.com/comparacion/espana_vs_alemania/EUR</loc>
</url>
<url>
<loc>https://preciosmundi.com/comparacion/argentina_vs_chile/USD</loc>
</url>
<url>
<loc>https://preciosmundi.com/comparacion/venezuela_vs_ecuador/USD</loc>
</url>
<url>
<loc>https://preciosmundi.com/comparacion/mexico_vs_estados-

Vemos que las páginas web van entre dos etiquetas `<loc> </loc>` y dos etiquetas `<url> </url>`

In [4]:
urlTags = soup.find_all("loc") #Buscamos todos los loc
print("En el sitio web hay un total de {0} URLs".format(len(urlTags)))

En el sitio web hay un total de 998 URLs


In [5]:
urlTags

[<loc>https://preciosmundi.com/</loc>,
 <loc>https://preciosmundi.com/europa/</loc>,
 <loc>https://preciosmundi.com/america/</loc>,
 <loc>https://preciosmundi.com/africa/</loc>,
 <loc>https://preciosmundi.com/asia/</loc>,
 <loc>https://preciosmundi.com/oceania/</loc>,
 <loc>https://preciosmundi.com/comparar</loc>,
 <loc>https://preciosmundi.com/comparacion/espana_vs_alemania/EUR</loc>,
 <loc>https://preciosmundi.com/comparacion/argentina_vs_chile/USD</loc>,
 <loc>https://preciosmundi.com/comparacion/venezuela_vs_ecuador/USD</loc>,
 <loc>https://preciosmundi.com/comparacion/mexico_vs_estados-unidos/USD</loc>,
 <loc>https://preciosmundi.com/comparacion/argentina_vs_brasil/USD</loc>,
 <loc>https://preciosmundi.com/comparacion/espana_vs_italia/EUR</loc>,
 <loc>https://preciosmundi.com/comparacion/colombia_vs_venezuela/USD</loc>,
 <loc>https://preciosmundi.com/comparacion/espana_vs_francia/EUR</loc>,
 <loc>https://preciosmundi.com/ranking/los-salarios-mas-altos-y-mas-bajos-de-asia</loc>,
 <

Ahora obtenemos el texto que está entre las dos etiquetas (las diferentes URL) y lo guardamos en una lista.

In [6]:
url_list=[]

for urls in urlTags:
    url_list.append(urls.get_text()) #Extraemos el texto de las tags

Veamos los primeros 20 elementos:

In [7]:
url_list[1:21]

['https://preciosmundi.com/europa/',
 'https://preciosmundi.com/america/',
 'https://preciosmundi.com/africa/',
 'https://preciosmundi.com/asia/',
 'https://preciosmundi.com/oceania/',
 'https://preciosmundi.com/comparar',
 'https://preciosmundi.com/comparacion/espana_vs_alemania/EUR',
 'https://preciosmundi.com/comparacion/argentina_vs_chile/USD',
 'https://preciosmundi.com/comparacion/venezuela_vs_ecuador/USD',
 'https://preciosmundi.com/comparacion/mexico_vs_estados-unidos/USD',
 'https://preciosmundi.com/comparacion/argentina_vs_brasil/USD',
 'https://preciosmundi.com/comparacion/espana_vs_italia/EUR',
 'https://preciosmundi.com/comparacion/colombia_vs_venezuela/USD',
 'https://preciosmundi.com/comparacion/espana_vs_francia/EUR',
 'https://preciosmundi.com/ranking/los-salarios-mas-altos-y-mas-bajos-de-asia',
 'https://preciosmundi.com/ranking/los-salarios-mas-altos-y-mas-bajos-de-europa',
 'https://preciosmundi.com/ranking/los-salarios-mas-altos-y-mas-bajos-de-america',
 'https://p

Los 17 primeros elementos de la listan no nos interesan para este trabajo en particular, dado que solo queremos obtener tablas de precios en los distintos países. Por tanto, los eliminamos.

In [8]:
del url_list[0:18]

Veamos los primeros 21 elementos:

In [9]:
url_list[0:21]

['https://preciosmundi.com/afganistan/',
 'https://preciosmundi.com/afganistan/precios-supermercado',
 'https://preciosmundi.com/afganistan/precio-restaurantes',
 'https://preciosmundi.com/afganistan/precio-ropa-calzado',
 'https://preciosmundi.com/afganistan/precio-transporte-servicios',
 'https://preciosmundi.com/afganistan/precio-vivienda-salarios',
 'https://preciosmundi.com/afganistan/precio-ocio-deportes',
 'https://preciosmundi.com/albania/',
 'https://preciosmundi.com/albania/precios-supermercado',
 'https://preciosmundi.com/albania/precio-restaurantes',
 'https://preciosmundi.com/albania/precio-ropa-calzado',
 'https://preciosmundi.com/albania/precio-transporte-servicios',
 'https://preciosmundi.com/albania/precio-vivienda-salarios',
 'https://preciosmundi.com/albania/precio-ocio-deportes',
 'https://preciosmundi.com/alemania/',
 'https://preciosmundi.com/alemania/precios-supermercado',
 'https://preciosmundi.com/alemania/precio-restaurantes',
 'https://preciosmundi.com/aleman

Podemos ver que cada país tiene 7 páginas web diferentes donde está almacenada la información. El objetivo de ese trabajo es extraer los datos de precios de vivienda para todos los países que aparecen en esta página web.

In [10]:
len(url_list)

980

Existen 980 elementos en la lista, divididos en 140 grupos de 7 elementos cada uno (País+super+restaurante+ropa+calzado+transporte+vivienda+ocio). Por lo tanto, hay 140 países diferentes. Seleccionamos sólo las url de precios de vivienda, aprovechando que están estructurados de manera periódica.

In [11]:
url_subList = [url_list[n+5] for n in range(0, len(url_list), 7)] 
#Como todos los países tienen las mismas páginas y hay 140 países, podemos coger la quinta página de cada grupo de 7

In [12]:
url_subList

['https://preciosmundi.com/afganistan/precio-vivienda-salarios',
 'https://preciosmundi.com/albania/precio-vivienda-salarios',
 'https://preciosmundi.com/alemania/precio-vivienda-salarios',
 'https://preciosmundi.com/andorra/precio-vivienda-salarios',
 'https://preciosmundi.com/angola/precio-vivienda-salarios',
 'https://preciosmundi.com/arabia-saudita/precio-vivienda-salarios',
 'https://preciosmundi.com/argelia/precio-vivienda-salarios',
 'https://preciosmundi.com/argentina/precio-vivienda-salarios',
 'https://preciosmundi.com/armenia/precio-vivienda-salarios',
 'https://preciosmundi.com/australia/precio-vivienda-salarios',
 'https://preciosmundi.com/austria/precio-vivienda-salarios',
 'https://preciosmundi.com/azerbaiyan/precio-vivienda-salarios',
 'https://preciosmundi.com/bahrein/precio-vivienda-salarios',
 'https://preciosmundi.com/bangladesh/precio-vivienda-salarios',
 'https://preciosmundi.com/belgica/precio-vivienda-salarios',
 'https://preciosmundi.com/bielorrusia/precio-vivi

Podemos pasar entonces a la siguiente parte del trabajo: La obtención de los datos contenidos en esas páginas web. El código que emplearemos será la siguiente función.

In [13]:
def get_realstate(url):
    headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\*/*;q=0.8", 
           "Accept-Encoding": "gzip, deflate, sdch, br",
           "Accept-Language": "en-US,en;q=0.8",
           "Cache-Control": "no-cache",
           "dnt": "1",
           "Pragma": "no-cache",
           "Upgrade-Insecure-Requests": "1",
           "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
    }
    r_real = requests.get(url,headers=headers) #Obtenemos la página web
    html_real = r_real.text
    soup_real = BeautifulSoup(html_real,'html') #Tranformamos en objeto soup
    data_url = soup_real.find('div',{'class':'table-responsive-md'}) #Buscamos la tabla, esto lo hicimos con ayuda de un navegador
    country=url.replace('/', ' ').split()[2].capitalize() #Cogemos el país de la url y le ponemos la primera letra mayúscula
    columns=['Comprar vivienda en las afueras de la ciudad (precio por m2)',
       'Comprar vivienda en el centro de la ciudad (precio por m2)',
       'Vivienda (3 habitaciones) en las afueras',
       'Vivienda (3 habitaciones) en centro de la ciudad',
       'Apartamento (1 dormitorio) en las afueras',
       'Apartamento (1 dormitorio) en el centro de la ciudad'] #Nombres de las columnas de precios de vivienda
    if data_url != None: #En el caso de que la página web tenga una tabla hacemos esto
        real_state_table= str(data_url)
        df_temp = pd.read_html(real_state_table)
        df_temp = df_temp[0]
        df_temp['DÃ³lar ($)'] = df_temp['DÃ³lar ($)'].str.replace('$', '') #Cambios de estilo en el dataframe
        df_temp['DÃ³lar ($)'] = df_temp['DÃ³lar ($)'].str.replace(',', '.')
        df_temp['DÃ³lar ($)'] = df_temp['DÃ³lar ($)'].astype(float)
        selected_columns = df_temp[['Producto','DÃ³lar ($)']] #Mantenemos sólo el precio en dólares
        df_temp2 = selected_columns.copy()
        df_temp2.columns = ['Producto','Precio en dólares'] #Cambiamos el nombre para que quede más presentable
        df_temp3=df_temp2.transpose() #Queremos cada país como una fila, y esta al revés. Transponemos
        df_temp3.columns=list(df_temp3.iloc[0]) #Al trasponer el nombre de las columnas es una fila del dataframe
        df_temp3=df_temp3.drop(df_temp3.index[0]) #Lo solucionamos y guardamos el nombre
        df_temp4 = df_temp3.rename(index={'Precio en dólares':country}) #Renombramos la fila al nombre del país
        time.sleep(0.5) #Para evitar hacer muchas peticiones al sitio, obligamos a parar 0.5 segundos después de cada petición.
        return(df_temp4)
    else: #Si no tiene tabla creamos un dataframe con nans para retornarlo
        print('Sin información de precios de vivienda en '+ country)
        df_null=pd.DataFrame(np.zeros([1, 6])*np.nan).rename(index={0:country})
        df_null.columns=columns
        return(df_null)

Recorremos un bucle en el que para cada una de las páginas web obtenidas se ejecute la función que hemos creado y extraiga los datos que necesitamos. Esos datos se concatenarán al dataframe principal. Existen casos de falta de información parcial o total a los que se asignarán NaN si no se encuentra el valor que buscamos.

In [14]:
df_real_state = pd.DataFrame() 

for url in url_subList:
    df_temp=get_realstate(url)
    df_real_state=pd.concat([df_real_state,df_temp])

Sin información de precios de vivienda en Botswana
Sin información de precios de vivienda en Togo
Sin información de precios de vivienda en Venezuela


Renombramos las columnas con un nombre más acorde que explique sus contenidos:

In [15]:
df_real_state.columns=['Vivienda a las afueras, compra (USD/m2)',
                       'Vivienda en el centro de la ciudad, compra (USD/m2)',
                       'Vivienda (3 habitaciones) a las afueras, alquiler (USD)',
                       'Vivienda (3 habitaciones) en el centro de la ciudad, alquiler (USD)',
                       'Apartamento (1 dormitorio) en las afueras, alquiler (USD)',
                       'Apartamento (1 dormitorio) en el centro de la ciudad, alquiler (USD)']

Aquí podemos ver el dataframe:

In [16]:
df_real_state

,"Vivienda a las afueras, compra (USD/m2)","Vivienda en el centro de la ciudad, compra (USD/m2)","Vivienda (3 habitaciones) a las afueras, alquiler (USD)","Vivienda (3 habitaciones) en el centro de la ciudad, alquiler (USD)","Apartamento (1 dormitorio) en las afueras, alquiler (USD)","Apartamento (1 dormitorio) en el centro de la ciudad, alquiler (USD)"
Afganistan,374.75,788.54,153.55,266.75,71.57,132.73
Albania,779.89,1519.38,314.78,526.19,184.17,298.8
Alemania,4087.12,5955.52,1284.52,1634.85,653.94,864.13
Andorra,3269.7,3853.57,980.91,1401.3,572.2,829.1
Angola,3999.96,5999.95,9000.07,9979.32,88.35,171.6
...,...,...,...,...,...,...
Venezuela,NaN,NaN,NaN,NaN,NaN,NaN
Vietnam,1150.15,2392.44,589.38,898.13,299.16,419.95
Yemen,752.15,1450.37,243.66,425.8,103.85,183.74
Zambia,27921.6,17008.2,281.44,481.41,121.49,139.5


Exportamos a un csv:

In [17]:
df_real_state.to_csv('PreciosViviendaPorPais.csv')